In [1]:
import pandas as pd
import numpy as np

# Graph
import plotly.express as px
import matplotlib.pyplot as plt

# Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import callbacks

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# tf
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import SGD

import warnings
warnings.filterwarnings("ignore") 

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV 
from sklearn.model_selection import cross_val_score 

In [2]:
path = "../../../dataset_clean.csv"
TARGET = 'Label'

In [3]:
df = pd.read_csv(path)

In [4]:
df = df[['Init_Win_bytes_forward',
'Total Length of Fwd Packets',
'Bwd Header Length',
'Destination Port',
'Subflow Fwd Bytes',
'Packet Length Std',
'Packet Length Variance',
'Bwd Packets/s',
'Average Packet Size',
'Bwd Packet Length Std',
         "Label"]]

In [5]:
y = df[TARGET]
X = df.drop([TARGET],axis=1)

In [6]:
y = y.to_numpy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X.shape[1]))

# Model

## Cross Validation

In [9]:
## modèle de base pour la recherche des paramètres optimaux 

def create_model(activation='relu', number_of_neurons=256, number_of_layers=2, init_mode='uniform'):
    model = Sequential()
    model.add(GRU(32, input_shape=(1, 10),kernel_initializer=init_mode, activation=activation))
    for i in range(number_of_layers):
        model.add(Dense(number_of_neurons, activation=activation))
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    
    return model

skmodel = KerasClassifier(build_fn=create_model) 

params = {'activation':['relu', 'tanh'], 
          'batch_size':[4, 8, 16, 32, 64],
          'epochs':[20, 50, 100, 200, 500],
          'init_mode':['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
          'number_of_layers':[3,5,7,9,11],
          'number_of_neurons':[8,16,32,64,128,256]
}

random_search = RandomizedSearchCV(skmodel, param_distributions=params, cv=5, n_jobs=-1)

random_search_results = random_search.fit(X_train, y_train, verbose=0) 
print("Best: {} using {}".format(np.round(random_search_results.best_score_,4), random_search_results.best_params_))

Best: 0.9791 using {'number_of_neurons': 32, 'number_of_layers': 7, 'init_mode': 'normal', 'epochs': 500, 'batch_size': 32, 'activation': 'relu'}


In [14]:
model = Sequential()

model.add(GRU(32, input_shape=(1,10), activation="relu", return_sequences=True,  kernel_initializer='normal'))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(y.shape[1], activation='sigmoid'))
model.add(Flatten())

model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 1, 32)             4224      
                                                                 
 dense_16 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_17 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_18 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_19 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_20 (Dense)            (None, 1, 32)             1056      
                                                                 
 dense_21 (Dense)            (None, 1, 32)            

In [9]:
model = Sequential()

model.add(GRU(220, input_shape=(1,10), activation="relu", return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(220,activation="relu"))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 220)            153120    
                                                                 
 dropout (Dropout)           (None, 1, 220)            0         
                                                                 
 gru_1 (GRU)                 (None, 220)               291720    
                                                                 
 dense (Dense)               (None, 2)                 442       
                                                                 
Total params: 445,282
Trainable params: 445,282
Non-trainable params: 0
_________________________________________________________________


In [10]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", 
                                        patience = 3, 
                                        restore_best_weights = True)

In [15]:
history = model.fit(X_train, y_train, 
                    epochs = 500, 
                    validation_data= (X_test, y_test),
                    batch_size=32
                    )
history

Epoch 1/500
619/619 [==============================] - 3s 2ms/step - loss: 0.1742 - accuracy: 0.9503 - val_loss: 0.1181 - val_accuracy: 0.9638
Epoch 2/500
619/619 [==============================] - 1s 2ms/step - loss: 0.1161 - accuracy: 0.9598 - val_loss: 0.1065 - val_accuracy: 0.9646
Epoch 3/500
619/619 [==============================] - 1s 2ms/step - loss: 0.1037 - accuracy: 0.9607 - val_loss: 0.0943 - val_accuracy: 0.9651
Epoch 4/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0990 - accuracy: 0.9605 - val_loss: 0.0929 - val_accuracy: 0.9663
Epoch 5/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0953 - accuracy: 0.9623 - val_loss: 0.0878 - val_accuracy: 0.9659
Epoch 6/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0949 - accuracy: 0.9632 - val_loss: 0.0920 - val_accuracy: 0.9655
Epoch 7/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0943 - accuracy: 0.9636 - val_loss: 0.0954 - val_accuracy: 0.9665

Epoch 58/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0754 - accuracy: 0.9688 - val_loss: 0.0791 - val_accuracy: 0.9717
Epoch 59/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0734 - accuracy: 0.9696 - val_loss: 0.0785 - val_accuracy: 0.9721
Epoch 60/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0726 - accuracy: 0.9696 - val_loss: 0.0807 - val_accuracy: 0.9703
Epoch 61/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0761 - accuracy: 0.9686 - val_loss: 0.0820 - val_accuracy: 0.9715
Epoch 62/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0739 - accuracy: 0.9688 - val_loss: 0.0830 - val_accuracy: 0.9715
Epoch 63/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0731 - accuracy: 0.9694 - val_loss: 0.0783 - val_accuracy: 0.9727
Epoch 64/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0731 - accuracy: 0.9697 - val_loss: 0.0862 - val_accuracy:

Epoch 115/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0671 - accuracy: 0.9712 - val_loss: 0.0834 - val_accuracy: 0.9715
Epoch 116/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0688 - accuracy: 0.9706 - val_loss: 0.0739 - val_accuracy: 0.9723
Epoch 117/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0684 - accuracy: 0.9710 - val_loss: 0.0730 - val_accuracy: 0.9727
Epoch 118/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0658 - accuracy: 0.9708 - val_loss: 0.0764 - val_accuracy: 0.9701
Epoch 119/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0677 - accuracy: 0.9707 - val_loss: 0.0755 - val_accuracy: 0.9729
Epoch 120/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0684 - accuracy: 0.9704 - val_loss: 0.0699 - val_accuracy: 0.9727
Epoch 121/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0672 - accuracy: 0.9711 - val_loss: 0.0768 - val_ac

619/619 [==============================] - 1s 2ms/step - loss: 0.0593 - accuracy: 0.9733 - val_loss: 0.0741 - val_accuracy: 0.9731
Epoch 172/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0612 - accuracy: 0.9722 - val_loss: 0.0646 - val_accuracy: 0.9743
Epoch 173/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0598 - accuracy: 0.9734 - val_loss: 0.0726 - val_accuracy: 0.9723
Epoch 174/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0606 - accuracy: 0.9726 - val_loss: 0.0765 - val_accuracy: 0.9737
Epoch 175/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0600 - accuracy: 0.9722 - val_loss: 0.0653 - val_accuracy: 0.9733
Epoch 176/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0607 - accuracy: 0.9723 - val_loss: 0.0672 - val_accuracy: 0.9741
Epoch 177/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0587 - accuracy: 0.9736 - val_loss: 0.0671 - val_accuracy: 0.9804

619/619 [==============================] - 1s 2ms/step - loss: 0.0610 - accuracy: 0.9728 - val_loss: 0.0751 - val_accuracy: 0.9713
Epoch 228/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0570 - accuracy: 0.9731 - val_loss: 0.0730 - val_accuracy: 0.9741
Epoch 229/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0586 - accuracy: 0.9735 - val_loss: 0.0664 - val_accuracy: 0.9723
Epoch 230/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0573 - accuracy: 0.9732 - val_loss: 0.0635 - val_accuracy: 0.9747
Epoch 231/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0540 - accuracy: 0.9747 - val_loss: 0.0612 - val_accuracy: 0.9796
Epoch 232/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0529 - accuracy: 0.9749 - val_loss: 0.0674 - val_accuracy: 0.9723
Epoch 233/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0587 - accuracy: 0.9740 - val_loss: 0.0674 - val_accuracy: 0.9723

619/619 [==============================] - 1s 2ms/step - loss: 0.0518 - accuracy: 0.9767 - val_loss: 0.0674 - val_accuracy: 0.9739
Epoch 284/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0543 - accuracy: 0.9758 - val_loss: 0.0645 - val_accuracy: 0.9737
Epoch 285/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0570 - accuracy: 0.9741 - val_loss: 0.0646 - val_accuracy: 0.9741
Epoch 286/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0519 - accuracy: 0.9758 - val_loss: 0.0640 - val_accuracy: 0.9731
Epoch 287/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0520 - accuracy: 0.9758 - val_loss: 0.0725 - val_accuracy: 0.9737
Epoch 288/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0534 - accuracy: 0.9760 - val_loss: 0.0621 - val_accuracy: 0.9764
Epoch 289/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0604 - accuracy: 0.9749 - val_loss: 0.0758 - val_accuracy: 0.9749

619/619 [==============================] - 1s 2ms/step - loss: 0.0530 - accuracy: 0.9744 - val_loss: 0.0680 - val_accuracy: 0.9745
Epoch 340/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0532 - accuracy: 0.9760 - val_loss: 0.0646 - val_accuracy: 0.9752
Epoch 341/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0536 - accuracy: 0.9770 - val_loss: 0.0627 - val_accuracy: 0.9741
Epoch 342/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0573 - accuracy: 0.9736 - val_loss: 0.0621 - val_accuracy: 0.9754
Epoch 343/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0517 - accuracy: 0.9763 - val_loss: 0.0570 - val_accuracy: 0.9846
Epoch 344/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0493 - accuracy: 0.9772 - val_loss: 0.0589 - val_accuracy: 0.9747
Epoch 345/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0495 - accuracy: 0.9778 - val_loss: 0.0636 - val_accuracy: 0.9752

619/619 [==============================] - 1s 2ms/step - loss: 0.0528 - accuracy: 0.9747 - val_loss: 0.0677 - val_accuracy: 0.9772
Epoch 396/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0561 - accuracy: 0.9730 - val_loss: 0.0838 - val_accuracy: 0.9610
Epoch 397/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0524 - accuracy: 0.9749 - val_loss: 0.0617 - val_accuracy: 0.9749
Epoch 398/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0515 - accuracy: 0.9761 - val_loss: 0.0707 - val_accuracy: 0.9693
Epoch 399/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0533 - accuracy: 0.9740 - val_loss: 0.0648 - val_accuracy: 0.9756
Epoch 400/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0516 - accuracy: 0.9749 - val_loss: 0.0632 - val_accuracy: 0.9842
Epoch 401/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0513 - accuracy: 0.9753 - val_loss: 0.0778 - val_accuracy: 0.9618

619/619 [==============================] - 1s 2ms/step - loss: 0.0516 - accuracy: 0.9769 - val_loss: 0.0693 - val_accuracy: 0.9723
Epoch 452/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0504 - accuracy: 0.9777 - val_loss: 0.0584 - val_accuracy: 0.9818
Epoch 453/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0482 - accuracy: 0.9788 - val_loss: 0.0689 - val_accuracy: 0.9745
Epoch 454/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0496 - accuracy: 0.9775 - val_loss: 0.0630 - val_accuracy: 0.9741
Epoch 455/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0479 - accuracy: 0.9792 - val_loss: 0.0600 - val_accuracy: 0.9784
Epoch 456/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0532 - accuracy: 0.9775 - val_loss: 0.0577 - val_accuracy: 0.9798
Epoch 457/500
619/619 [==============================] - 1s 2ms/step - loss: 0.0484 - accuracy: 0.9778 - val_loss: 0.0673 - val_accuracy: 0.9731

In [16]:
accuracy = model.evaluate(X_test, y_test)
accuracy

155/155 [==============================] - 0s 997us/step - loss: 0.0655 - accuracy: 0.9768


[0.06549247354269028, 0.9767676591873169]